In [6]:
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [5]:
userid = 'A00'
username = '투자자0'
current_date = '2020-01-10'
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}

In [8]:
instruments_m = pd.read_pickle('../../data/processed/instruments_m.pkl')